Preprocess the data Peter provided to put it into a ready to analyze form.  save it to the file noted below

In [1]:
output_filename = 'data/yields.parquet'

In [2]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np

In [3]:
parquet_file = '~/Downloads/merged_yield_df_area_planeted_harvested_winter_wheat_marked.parquet'
raw0 = pq.read_table(parquet_file).to_pandas()
raw0['Acres Harvested'] = raw0['Acres Harvested'].str.replace(',', '').astype(float)
raw0['Acres Planted'] = raw0['Acres Planted'].str.replace(',', '').astype(float)

#rename columns called County as CountyName
raw0.rename(columns={'County':'CountyName'}, inplace=True)
# new column called County that is the concatenation of State and CountyName
raw0['County'] = raw0['State'] + '/' + raw0['CountyName']
# drop the columns State and CountyName
raw0.drop(columns=['State','CountyName'], inplace=True)
# drop county_name, county_number, state_county_code, state_code
raw0.drop(columns=['qa.landsat','x','y','county_name','county_number','state_county_code','state_code'], inplace=True)



print('raw0: {:,} rows'.format(len(raw0)))


raw0: 24,395,812 rows


In [7]:
# for each year, keep only rows where winter wheat is TRUE
l = []
for y in range(2013,2023):
    mask = (raw0['year'] == y) & (raw0['is_winter_wheat_'+str(y)])
    l.append(raw0[mask])
raw1 = pd.concat(l).sort_values('time')
raw1 = raw1.drop(columns=[c for c in raw1.columns if 'is_winter_wheat' in c])
print('raw1: {:,} rows'.format(len(raw1)))

raw1: 12,199,609 rows


In [19]:
drop_list = ['lat_x', 'lon_x', 'ndvi.sentinel2','qa.sentinel2','lat_y','lon_y']
raw2 = raw1.drop(columns=drop_list)

In [20]:
raw3 = raw2.drop(raw2[raw2['ndvi.landsat'] > 1.0].index)
print('raw3: {:,} rows'.format(len(raw3)))
raw4 = raw3.dropna(subset=['ndvi.landsat'])
print('raw4: {:,} rows'.format(len(raw4)))
raw5 = raw4.dropna(subset=['Yield (Bu/Acre)'])
print('raw5: {:,} rows'.format(len(raw5)))

raw3: 12,198,253 rows
raw4: 10,272,407 rows
raw5: 8,401,661 rows


In [22]:
raw5.head()

,time,point,ndvi.landsat,year,Yield (Bu/Acre),Acres Planted,Acres Harvested,County
1007869,2013-03-18,417,0.198277,2013,23.2,58200.0,42700.0,NEBRASKA/BANNER
10561125,2013-03-18,4347,0.278712,2013,36.5,55800.0,54000.0,COLORADO/MORGAN
21620278,2013-03-18,8866,0.215333,2013,23.1,99800.0,86600.0,NEBRASKA/KIMBALL
16872384,2013-03-18,6923,0.209268,2013,23.1,99800.0,86600.0,NEBRASKA/KIMBALL
3073069,2013-03-18,1286,0.378128,2013,27.5,153000.0,148800.0,COLORADO/ADAMS


In [23]:

raw6 = raw5[['time','year', 'County','point','ndvi.landsat', 'Yield (Bu/Acre)','Acres Planted', 'Acres Harvested']]
raw6.head()

,time,year,County,point,ndvi.landsat,Yield (Bu/Acre),Acres Planted,Acres Harvested
1007869,2013-03-18,2013,NEBRASKA/BANNER,417,0.198277,23.2,58200.0,42700.0
10561125,2013-03-18,2013,COLORADO/MORGAN,4347,0.278712,36.5,55800.0,54000.0
21620278,2013-03-18,2013,NEBRASKA/KIMBALL,8866,0.215333,23.1,99800.0,86600.0
16872384,2013-03-18,2013,NEBRASKA/KIMBALL,6923,0.209268,23.1,99800.0,86600.0
3073069,2013-03-18,2013,COLORADO/ADAMS,1286,0.378128,27.5,153000.0,148800.0


In [24]:
# save raw6 to disk
raw6.to_parquet(output_filename)

In [25]:
# print the number of unique Counties by year
for y in range(2013,2023):
    mask = (raw6['year'] == y)
    print('Number of unique counties in {}: {:,}'.format(y, len(raw6[mask]['County'].unique())))
    
print('Number of unique counties: {:,}'.format(len(raw6['County'].unique())))

Number of unique counties in 2013: 296
Number of unique counties in 2014: 329
Number of unique counties in 2015: 333
Number of unique counties in 2016: 328
Number of unique counties in 2017: 272
Number of unique counties in 2018: 274
Number of unique counties in 2019: 272
Number of unique counties in 2020: 313
Number of unique counties in 2021: 254
Number of unique counties in 2022: 471
Number of unique counties: 634
